Imports

In [12]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Dataset creation, scaling and manipulation

In [26]:
# Loading dataset
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
m, n_features = X.shape

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1234)

# Scale
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

# Numpy to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# Converting y's to column vector
y_train = y_train.reshape(y_train.shape[0], -1)
y_test = y_test.reshape(y_test.shape[0], -1)

Model

Loss and optimisation

Training loop